In [ ]:
# default_exp load_videos

# module name here

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
def test_hello():
    print("hello!")
    return "hello!"

In [ ]:
assert test_hello() == "hello!"

hello!


In [ ]:
from nbdev.export import *
notebook2script()

Converted 00_load_videos.ipynb.
Converted index.ipynb.


Requires ffmpeg and youtube-dl to be installed to download videos from youtube live streams.

In [1]:
# "Shirahama Ohama Beach" 白浜大浜海水浴場（静岡県下田市）
!youtube-dl -f "bestvideo[height<=?1080]+bestaudio/best" -o "tmp.mp4" https://www.youtube.com/watch?v=WbRbWJZ7ZH0

[youtube] WbRbWJZ7ZH0: Downloading webpage
ERROR: This video is unavailable.


In [3]:
# 新舞子海水浴場 福ヶ浜 ライブ配信
!youtube-dl -f "bestvideo[height<=?1080]+bestaudio/best" -o "tmp.mp4" https://www.youtube.com/watch?v=3dkBkAjNay4

[youtube] 3dkBkAjNay4: Downloading webpage
[youtube] 3dkBkAjNay4: Downloading m3u8 information
[youtube] 3dkBkAjNay4: Downloading MPD manifest
[download] Destination: tmp.mp4
ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable